In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels import PanelOLS, RandomEffects, PooledOLS
from linearmodels.panel import compare
from scipy import stats

In [3]:
url = 'https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv'
data = pd.read_csv(url)

In [ ]:
data_panel = data.dropna(subset=['iso_code'])  ## paises individuais possuem iso_code

## sem o ISO code não existe a identificação da unidade no painel

## Assim, não é possível distinguir os países

In [ ]:
data_panel['energy_per_capita'] = data_panel['primary_energy_consumption'] / data_panel['population']  
## criada, pois é uma variável chave na relação entre crescimento econômico e emissões de carbono
## o crescimento econômico históricamente demanda um maior consumo energético

C:\Users\Ryan\AppData\Local\Temp\ipykernel_5800\3093921456.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_panel['energy_per_capita'] = data_panel['primary_energy_consumption'] / data_panel['population']  ## criada, pois é uma variável chave na relação entre crescimento econômico e emissões de carbono


In [6]:
for col in data_panel.columns:
    if col == "energy_per_capita":
        print(col)

data_panel['energy_per_capita']   ## verificando a criação da coluna

energy_per_capita


0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
           ...   
50406    0.000002
50407    0.000002
50408    0.000004
50409    0.000003
50410         NaN
Name: energy_per_capita, Length: 42480, dtype: float64

In [ ]:
data_panel["gdp_per_capita"] = data_panel["gdp"]/data_panel["population"]  ## a curva de kuznets exige metrica 
##per capita para testaR o efeito da riqueza média da população no meio ambiente

C:\Users\Ryan\AppData\Local\Temp\ipykernel_5800\2010020605.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_panel["gdp_per_capita"] = data_panel["gdp"]/data_panel["population"]  ## a curva de kuznets exige metrica per capita para testa o efeito da riqueza média da população no meio ambiente


Curva de Kuznets demonstra que a desigualdade social aumenta na industrialização e depois diminui.

Segunda ela, a classe média se forma a medida que a renda per capita cresce

In [ ]:
variaveis_interesse = ['iso_code', 'year', 'co2_per_capita', 'gdp_per_capita', 'energy_per_capita']
df = data_panel[variaveis_interesse].copy()

Para variáveis de identificação - iso_code, year - Utilizadas para a estruturação dos dados em painel

Variáveis de teste e controle - co2_per_capita, gdp_per_capita, energy_per_capita

In [9]:
df = df.dropna(subset=['co2_per_capita', 'gdp_per_capita', 'energy_per_capita'])

In [10]:
for col in ['co2_per_capita', 'gdp_per_capita', 'energy_per_capita']:
    df = df[df[col] > 0]            ## locando os valores como positivos                                            
    df[f'ln_{col}'] = np.log(df[col]) ## aplicando a transformação logarítmica

In [11]:
df['ln_gdp_sq'] = df['ln_gdp_per_capita']**2  ## variável quadrática do PIB

In [12]:
df = df.set_index(['iso_code', 'year'])  ## necessário configurar o índice para painel transnformando o dataframe em um conjunto de dados para painel

In [13]:
df

co2_per_capita  gdp_per_capita  energy_per_capita  \
iso_code year                                                      
AFG      1980           0.133     1164.057381       4.811944e-07   
         1981           0.166     1310.595291       6.106762e-07   
         1982           0.191     1453.903877       7.177443e-07   
         1983           0.231     1534.654334       9.051121e-07   
         1984           0.252     1525.631306       8.873819e-07   
...                       ...             ...                ...   
ZWE      2018           0.746     1781.658724       2.897943e-06   
         2019           0.672     1646.637118       2.603891e-06   
         2020           0.547     1492.810966       2.163087e-06   
         2021           0.647     1591.424869       2.382445e-06   
         2022           0.761     1611.891689       3.633069e-06   

               ln_co2_per_capita  ln_gdp_per_capita  ln_energy_per_capita  \
iso_code year                                                               
AFG      1980          -2.017406           7.059667            -14.546994   
         1981          -1.795767           7.178237            -14.308699   
         1982          -1.655482           7.282008            -14.147153   
         1983          -1.465338           7.336060            -13.915207   
         1984          -1.378326           7.330164            -13.934990   
...                          ...                ...                   ...   
ZWE      2018          -0.293030           7.485300            -12.751509   
         2019          -0.397497           7.406490            -12.858504   
         2020          -0.603306           7.308416            -13.043974   
         2021          -0.435409           7.372385            -12.947383   
         2022          -0.273122           7.385164            -12.525433   

               ln_gdp_sq  
iso_code year             
AFG      1980  49.838897  
         1981  51.527083  
         1982  53.027634  
         1983  53.817783  
         1984  53.731298  
...                  ...  
ZWE      2018  56.029717  
         2019  54.856100  
         2020  53.412947  
         2021  54.352061  
         2022  54.540643  

[7757 rows x 7 columns]

In [14]:
Y = df['ln_co2_per_capita']
X = df[['ln_gdp_per_capita', 'ln_gdp_sq', 'ln_energy_per_capita']]  ## definindo as variáveis preditoras e target

In [15]:
X_pooled = sm.add_constant(X, prepend=False) ## adicionando o intercepto (quando todos os valores x são zero) - serve para ajustar o nível geral da regressão

In [ ]:
model_pooled_sm = sm.OLS(Y, X_pooled).fit() ## pooled ols não considera as características próprias de cada país
model_fe = PanelOLS(Y, X, entity_effects=True).fit() ## efeitos fixos
model_re = RandomEffects(Y, X).fit() ## efeitos individuais são aleatórios e não correlacionados com as var explicativas

## o grande "vilão" do pooled OLS é a heterogeneidade das variáveis explicativas e a correlação entre ela
## e as variáveis



In [17]:

print("\n--- Resultados Pooled OLS (Modelo A) ---")
print(model_pooled_sm.summary())
print("\n--- Resultados Efeitos Fixos (Modelo B) ---")
print(model_fe)
print("\n--- Resultados Efeitos Aleatórios (Modelo C) ---")
model_re


--- Resultados Pooled OLS (Modelo A) ---
                            OLS Regression Results                            
Dep. Variable:      ln_co2_per_capita   R-squared:                       0.946
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                 4.548e+04
Date:                Thu, 04 Dec 2025   Prob (F-statistic):               0.00
Time:                        18:03:27   Log-Likelihood:                -3700.1
No. Observations:                7757   AIC:                             7408.
Df Residuals:                    7753   BIC:                             7436.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

Dep. Variable:,ln_co2_per_capita,R-squared:,0.7389
Estimator:,RandomEffects,R-squared (Between):,0.9498
No. Observations:,7757,R-squared (Within):,0.6995
Date:,"Thu, Dec 04 2025",R-squared (Overall):,0.9373
Time:,18:03:27,Log-likelihood,-84.012
Cov. Estimator:,Unadjusted,,
,,F-statistic:,7313.8
Entities:,164,P-value,0.0000
Avg Obs:,47.299,Distribution:,"F(3,7754)"
Min Obs:,17.000,,
Max Obs:,58.000,F-statistic (robust):,7313.8


In [ ]:
import numpy as np
from scipy.stats import chi2

b_fe = model_fe.params  ## coeficientes
b_re = model_re.params

# 2. Selecionar apenas parâmetros comuns
common = b_fe.index.intersection(b_re.index)

b_fe = b_fe[common]
b_re = b_re[common]

v_fe = model_fe.cov.loc[common, common]
v_re = model_re.cov.loc[common, common]

M = v_fe - v_re

# 5. Diferença dos coeficientes
q = b_fe - b_re

# 6. Estatística de Hausman
H = q.T @ np.linalg.inv(M) @ q

df = len(q)
p_value = 1 - chi2.cdf(H, df)

print("Estatística de Hausman:", H)
print("p-valor:", p_value)


Estatística de Hausman: 97.52588185308964
p-valor: 0.0


A hipótese nula de que os RE (estimadores de efeitos aleatórios) são consistentes será rejeitada.

Efeitos fixos são preferíveis, pois remove a heterogeneidade de cada país, correlacionada com as variáveis

Os fatores mencionados (insituições, dependência dos combustíveis fósseis e sustentabilidade) são fixas no tempo e possuem maior correlação com o PIB.